In [1]:
import numpy as np
import torch
import torch.nn as nn
import matplotlib.pyplot as plt
from netCDF4 import Dataset
from tqdm import tqdm
import torch.nn.functional as F
from sklearn.metrics import r2_score as R2
from sklearn.model_selection import KFold
from sklearn.metrics import r2_score as r2
from copy import deepcopy
import utils
from unet import UNet_nobatchnorm
from scipy.stats import pearsonr
#JU's addtion to automate inputs and outputs
import helper_functions as hf
import os
from data_loading import load_data_from_nc, degrade_space_gaussian, degrade_time_uniform 
from scipy.signal import convolve2d, convolve


def save_fn(var_input_list, var_output_list):
    var_input_join  = '_and_'.join(var_input_list)
    var_output_join = '_and_'.join(var_output_list)
    return '{}_to_{}'.format(var_input_join, var_output_join)

torch.cuda.set_device(0)
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print ('Running on ', device)

Running on  cuda:0


In [2]:
print(torch.__version__)
print(torch.version.cuda)

1.13.0
11.7


In [3]:
maxEpochs =  300 #small number is taken for debugging
nensemble = 10 #How many training sessions are run for each configuration 
Nbase = 16
decayunits=5 #The best SST satellite has a resolution of 9 km. It may be safe to set decayunits to be 20 km to be already resolvable by satellites

In [4]:
!nvidia-smi #GPU usage should be maxed out during training; tune batch_size according to that

Wed Jan 29 21:42:21 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 560.35.05              Driver Version: 560.35.05      CUDA Version: 12.6     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA A100-SXM4-40GB          Off |   00000000:C4:00.0 Off |                    0 |
| N/A   47C    P0             56W /  400W |       5MiB /  40960MiB |      0%      Default |
|                                         |                        |             Disabled |
+-----------------------------------------+-----

In [5]:
root_dir = '/work/uo0780/u241359/project_tide_synergy/data/'
nctrains, nctest = hf.load_data_from_nc_as_lists(root_dir)


Ntrain = np.sum([nc.dimensions['time_counter'].size for nc in nctrains], axis = 0)
Ntest = np.sum([nc.dimensions['time_counter'].size for nc in nctest], axis = 0)

print('number of training records:', Ntrain)
print('number of testing records:', Ntest)

numTrainFiles = len(nctrains)
numRecsFile = nctrains[0].dimensions['time_counter'].size #How many snapshots in time in each data set there is
print (numRecsFile)

# Modify the loadtrain function to pull data from preloaded memory
def loaddata_preloaded_train(index, batch_size, all_input_data, all_output_data):
    rec_slice = slice(index, index + batch_size)
    lim = 720
    width = 256
    yslice = slice(0, lim)
    xslice = slice(0, width)
    # print('rec_slice is:')
    # print(rec_slice)
    # print('mean of squared values of loaded input data:')
    # print("{0:0.32f}".format(np.nanmean(all_input_data[rec_slice, :, yslice, xslice]**2)))
    return (all_input_data[rec_slice, :, yslice, xslice], 
            all_output_data[rec_slice, :, yslice, xslice])
#Load test data as one single batch
def loaddata_preloaded_test(all_input_data, all_output_data):
    #rec_slice = slice(index, index + batch_size)
    lim = 720
    width = 256
    yslice = slice(0, lim)
    xslice = slice(0, width)
    # print('rec_slice is:')
    # print(rec_slice)
    # print('mean of squared values of loaded input data:')
    # print("{0:0.32f}".format(np.nanmean(all_input_data[rec_slice, :, yslice, xslice]**2)))
    return (all_input_data[:, :, yslice, xslice], 
            all_output_data[:, :, yslice, xslice])


number of training records: 600
number of testing records: 150
150


In [6]:
#Functions for low-pass filtering
def gaussian_kernel(decaylength): 
    """Generates a Gaussian kernel."""
    #decaylength is in the unit of grid resolution (4km in Aurelien's data.) So in physical units, the decay lenght would be decaylength*(4 km).
    size=int(2*decaylength)
    sigma=decaylength/(2*np.sqrt(2*np.log(2))) #Interpretting decaylength as the FWHM of Gaussian
    kernel = np.fromfunction(
        lambda x, y: (1 / np.sqrt(2 * np.pi * sigma ** 2)) * 
                      np.exp(-((x- size/2)**2 + (y-size/2)**2) / (2 * sigma ** 2)),
        (size, size)  
    ) #Creating a kernel with 
    return kernel / np.sum(kernel)  # Normalize the kernel
    
def degrade_space_gaussian(field, decaylength):
    nt, nx, ny = np.shape(field)
    kernel = gaussian_kernel(decaylength)
    filtered_field = np.empty([nt, nx, ny])

    for i in range(nt):
        filtered_field[i, : ,:] = convolve2d(field[i, : ,:], kernel, mode = 'same', boundary='symm')#,  fillvalue = np.average(field[i, : ,:]))
    return filtered_field

# Load all data into memory; no normalization is done here yet.
# Apply a spatial lowpass filter to the temperature field 'T_xy_ins'
# decayunits is how many units of grid spacing is the decay length scale. A grid spacing is 4km in Aurelien's data. So in physical units, the decay lenght would be decayunits*(4 km).
def preload_data_filterT(nctrains, total_records,decayunits=25,plot=True):
    #total_records = Ntrain#sum(nc.dimensions['time_counter'].size for nc in nctrains)
    #dimensions of data of the nc file.
    max_height = 722
    max_width = 258
    all_input_data = np.zeros((total_records, N_inp, max_height, max_width))*np.nan
    all_output_data = np.zeros((total_records, N_out, max_height, max_width))*np.nan
    current_index = 0
    for ncindex, ncdata in enumerate(nctrains):
        num_recs = ncdata.dimensions['time_counter'].size
        rec_slice = slice(current_index, current_index + num_recs)
        
        for ind, var_name in enumerate(var_input_names):
            data_slice = np.squeeze(ncdata.variables[var_name])
            # print('data_slice shape:')
            # print(data_slice.shape)        
            #Turns out to be (time, height, width)
            # print('var_name:')
            # print(var_name)
            # Apply lowpass filter when the field is 'T_xy_ins'
            if var_name == 'T_xy_ins':
                if plot == True:
                    #Plot an image before the filter
                    itime=20        
                    cmapmax=np.max(data_slice[itime,:,:])
                    cmapmin=np.min(data_slice[itime,:,:])
                    figT, axT = plt.subplots(1, 2, figsize=(5, 5))
                    figT.set_dpi(256)   
                    im0=axT[0].pcolor(data_slice[itime,:,:],vmin=cmapmin,vmax=cmapmax)
                    axT[0].set_aspect(1)
                #Lowpass filter
                data_slice=degrade_space_gaussian(data_slice,decayunits)
                if plot == True:
                    axT[1].pcolor(data_slice[itime,:,:],vmin=cmapmin,vmax=cmapmax)
                    axT[1].set_aspect(1)
                    cbar0=plt.colorbar(im0, ax=axT, fraction=0.046, pad=0.04)
            
            #For some variables, the dimensions in (x, y) may be smaller than (max_height, max_width). Changing the code so that it adapts them.
            # Get the actual dimensions of data_slice
            slice_height, slice_width = data_slice.shape[-2], data_slice.shape[-1]
            # Place data_slice into the corresponding slice of all_input_data
            all_input_data[rec_slice, ind, :slice_height, :slice_width] = data_slice
    

        for ind, var_name in enumerate(var_output_names):
            data_slice = np.squeeze(ncdata.variables[var_name])
            if var_name == 'T_xy_ins':
                if plot == True:
                    #Plot an image before the filter
                    itime=20        
                    cmapmax=np.max(data_slice[itime,:,:])
                    cmapmin=np.min(data_slice[itime,:,:])
                    figT, axT = plt.subplots(1, 2, figsize=(5, 5))
                    figT.set_dpi(256)   
                    im0=axT[0].pcolor(data_slice[itime,:,:],vmin=cmapmin,vmax=cmapmax)
                    axT[0].set_aspect(1)
                #Lowpass filter
                data_slice=degrade_space_gaussian(data_slice,decayunits)
                if plot == True:
                    axT[1].pcolor(data_slice[itime,:,:],vmin=cmapmin,vmax=cmapmax)
                    axT[1].set_aspect(1)
                    cbar0=plt.colorbar(im0, ax=axT, fraction=0.046, pad=0.04)
            #all_output_data[rec_slice, ind, :, :] = data_slice
            # Get the actual dimensions of data_slice
            slice_height, slice_width = data_slice.shape[-2], data_slice.shape[-1]
            # Place data_slice into the corresponding slice of all_input_data
            all_output_data[rec_slice, ind, :slice_height, :slice_width] = data_slice

        current_index += num_recs
        
    return all_input_data, all_output_data


In [7]:
def run_model(var_input_names, var_output_names, save_fn_prefix, N_inp, N_out, iensemble, R2_all, corr_all):
    def totorch(x):
        return torch.tensor(x, dtype = torch.float).cuda()

    model = UNet_nobatchnorm(N_inp, N_out, bilinear = True, Nbase = Nbase).cuda()
    #model = torch.compile(UNet(N_inp, N_out, bilinear = True, Nbase = Nbase).cuda())

    if iensemble == 0:
        input = torch.randn(1,N_inp,256,720).to(device) 
        output = model(input)
        print('Model has ', utils.nparams(model)/1e6, ' million params')

    # for index in range(0, Ntrain, batch_size):
    #     inp, out = loadtrain_preloaded(index, batch_size, all_train_input, all_train_output)
    #     print(inp.shape, out.shape)
#         print(np.nanmean(inp**2), np.max(inp**2), inp.shape)
#         print(np.nanmean(out**2), np.max(out**2), inp.shape)

    inp_test, out_test = loaddata_preloaded_test(all_test_input, all_test_output)
    #inp, out_test = loadtest()
    # print('shapes of input and output TEST data:')
    # print(inp_test.shape, out_test.shape)
    with torch.no_grad():
        inp_test = totorch(inp_test)

    Tcycle = 10
    criterion_train  = nn.L1Loss()
    optim = torch.optim.AdamW(model.parameters(), lr=lr0, betas=(0.9, 0.999), eps=1e-8, weight_decay=1e-5*100) #increase weight_decay ***

    r2_test = np.zeros(maxEpochs)
    epochmin = []
    maxr2l = []

    learn = np.zeros(maxEpochs)
    minloss = 1000
    maxR2 = -1000
    minlosscount = 0
    perm = False

    model_best = deepcopy(model)  # Initialize before the loop for safety

    #print('Starting training loop')
    for epoch in tqdm(range(maxEpochs)):
        lr = utils.cosineSGDR(optim, epoch, T0=Tcycle, eta_min=0, eta_max=lr0, scheme = 'constant')  #captioning this seems to make the printed corr larger??***
        model.train()
        index_perm = np.arange(0, Ntrain, batch_size)
        
        if perm:
            index_perm = np.random.permutation(index_perm)
        
        for index in index_perm:
            inp, out = loaddata_preloaded_train(index, batch_size, all_train_input, all_train_output)            
#           inp, out = loadtrain(index, batch_size, nctrains)
            inp, out = totorch(inp), totorch(out)
            #continue #do this to pause the later operations to check how long it takes for the steps up to this 
            out_mod = model(inp)
            loss = criterion_train(out.squeeze(), out_mod.squeeze())
            #Set gradient to zero
            optim.zero_grad()
            #Compute gradients       
            loss.backward()
            #Update parameters with new gradient
            optim.step()
            #Record train loss
            #scheduler.step()
          
        model.eval()
        with torch.no_grad():
            #model_cpu = model.to('cpu')
            #out_mod = model_cpu(inp_test.to('cpu'))
            out_mod=model(inp_test)
            
            r2 = R2(out_test.flatten(), (out_mod).cpu().numpy().flatten())
            r2_test[epoch] = r2
            #print('Debugging: R2 of current epoch:', r2)#Debugging
            #record current best model and best predictions
            if maxR2 <  r2:
                maxR2 = r2
                epochmin.append(epoch)
                maxr2l.append(maxR2)                
                model_best = deepcopy(model)
                corr, pval = pearsonr(out_test.flatten(), (out_mod).cpu().numpy().flatten())
                print('R2:', r2, ' corr: ', corr, ' pval: ', pval)
            #model = model_cpu.to(device)

    #_, out_test = loadtest()
    model_best.eval()
    with torch.no_grad():
    #     inp_test = totorch(inp)
        model_best.to('cpu') #added by HW 
        out_mod = model_best(inp_test.to('cpu')).detach().cpu().numpy()

    print('training finished')
    
    R2_all[iensemble]=R2(out_test.flatten(), out_mod.flatten())
    corr_all[iensemble]=pearsonr(out_test.flatten(), out_mod.flatten())[0]
    # print('Best model R2:', R2_all[iensemble])#pearsonr(out_test.flatten(), out_mod.flatten())[0])
    # print('Best model corr:', corr_all[iensemble])#R2(out_test.flatten(), out_mod.flatten()))
    print('start saving')
    # Nx, Ny = out_test.shape[2:]; Nx, Ny
    # print(out_mod.shape, 'outout model shape')
    dr = '/work/uo0780/u241359/project_tide_synergy/trainedmodels' #'./models/to_vel'
    os.makedirs(dr, exist_ok=True) # exist_ok=True allows the function to do nothing (i.e., not raise an error) if the directory already exists.
    fstr = f'{save_fn_prefix}_rp_{iensemble}'
    PATH = dr + f'/{fstr}.pth'
    torch.save(model_best.state_dict(), PATH)
    print('model saved')

In [8]:
vi1 = 'ssh_ins'
vi2 = 'T_xy_ins'

vo1 = 'ssh_cos'
vo2 = 'ssh_sin'

batch_size = 80 #maximizing it so that the GPU memory maxes out. Needs to be divisible by Ntrain. Otherwise there will be size mismatch issues.
lr0 = 0.005*10/batch_size #Roughly should scale inversely to batch_size

var_input_names = [vi1, vi2]
var_output_names = [vo1, vo2]
N_inp = len(var_input_names)
N_out = len(var_output_names)

save_fn_prefix  = 'any_{}{}_{}{}_nobatchnorm_degradeT_du_{}'.format(vi1, vi2, vo1, vo2, decayunits)
nctrains, nctest = hf.load_data_from_nc_as_lists(root_dir)

all_train_input, all_train_output = preload_data_filterT(nctrains, Ntrain, decayunits=decayunits,plot=False)
all_test_input, all_test_output = preload_data_filterT(nctest, Ntest, decayunits=decayunits,plot=False)

#Normalize data
#Compute mean and variance for normalization
mean_input=np.nanmean(np.concatenate((all_train_input, all_test_input), axis=0),axis=(0, 2, 3))
mean_output=np.nanmean(np.concatenate((all_train_output, all_test_output), axis=0),axis=(0, 2, 3))
#Subtract the data with their means
all_train_input=all_train_input-mean_input[None, :, None, None]
all_train_output=all_train_output-mean_output[None, :, None, None]
all_test_input=all_test_input-mean_input[None, :, None, None]
all_test_output=all_test_output-mean_output[None, :, None, None]
#Compute the variances
var_input=np.nanmean((np.concatenate((all_train_input, all_test_input), axis=0))**2,axis=(0, 2, 3))
var_output=np.nanmean((np.concatenate((all_train_output, all_test_output), axis=0))**2,axis=(0, 2, 3))
print("mean and variance of all input data, after lowpass is applied to Temperature:")
print(mean_input,var_input)
print("mean and variance of all output data, after lowpass is applied to Temperature:")
print(mean_output,var_output)

#Scale the data so that they have variance of 1
all_train_input=all_train_input/np.sqrt(var_input[None, :, None, None])
all_train_output=all_train_output/np.sqrt(var_output[None, :, None, None])
all_test_input=all_test_input/np.sqrt(var_input[None, :, None, None])
all_test_output=all_test_output/np.sqrt(var_output[None, :, None, None])
#Have checked that after these operations, the data is scaled to be zero mean and variance 1.

#Recording performance metrics on test data after eaching training cycle
R2_all = np.zeros(nensemble)
corr_all = np.zeros(nensemble)
for iensemble in np.arange(nensemble):
    run_model(var_input_names, var_output_names, save_fn_prefix, N_inp, N_out, iensemble, R2_all, corr_all)  
print('R2 from the best models in each run are:')
print(R2_all)
print('corr from the best models in each run are:')
print(corr_all)

mean and variance of all input data, after lowpass is applied to Temperature:
[ 0.03307104 25.14131891] [0.3119807  0.33904831]
mean and variance of all output data, after lowpass is applied to Temperature:
[-5.16228102e-04 -9.83592627e-05] [9.36516511e-05 1.01456128e-04]
Model has  1.124562  million params


  0%|          | 1/300 [00:06<32:48,  6.58s/it]

R2: -0.04348686068220031  corr:  0.001417861415834805  pval:  5.449011176673063e-26


  1%|          | 2/300 [00:12<30:52,  6.22s/it]

R2: -0.036596097739144406  corr:  0.06005894885697241  pval:  0.0


...

R2: 0.7437581719662725  corr:  0.8631771695579357  pval:  0.0


 47%|████▋     | 140/300 [12:09<14:27,  5.42s/it]

R2: 0.7453289228892956  corr:  0.8638740719876276  pval:  0.0


100%|██████████| 300/300 [24:59<00:00,  5.00s/it]


training finished
start saving
model saved
R2 from the best models in each run are:
[0.72735387 0.72171685 0.72828243 0.72888905 0.73266254 0.74324252
 0.73802416 0.73594124 0.72648617 0.74533718]
corr from the best models in each run are:
[0.85394472 0.84972726 0.85367671 0.85465628 0.85666538 0.86292938
 0.85969137 0.85806237 0.85382694 0.86387864]


In [9]:
print('R2 from the best models in each run are:')
print(R2_all)
print('corr from the best models in each run are:') 
print(corr_all)

R2 from the best models in each run are:
[0.72735387 0.72171685 0.72828243 0.72888905 0.73266254 0.74324252
 0.73802416 0.73594124 0.72648617 0.74533718]
corr from the best models in each run are:
[0.85394472 0.84972726 0.85367671 0.85465628 0.85666538 0.86292938
 0.85969137 0.85806237 0.85382694 0.86387864]


In [10]:
vi1 = 'T_xy_ins'
vi2 = 'u_xy_ins'
vi3 = 'v_xy_ins'

vo1 = 'ssh_cos'
vo2 = 'ssh_sin'


batch_size = 60 #maximizing it so that the GPU memory maxes out. Needs to be divisible by Ntrain. Otherwise there will be size mismatch issues.
lr0 = 0.005*10/batch_size #Roughly should scale inversely to batch_size


var_input_names = [vi1, vi2, vi3]
var_output_names = [vo1, vo2]
N_inp = len(var_input_names)
N_out = len(var_output_names)

save_fn_prefix  = 'any_{}{}{}_{}{}_nobatchnorm_degradeT_du_{}'.format(vi1, vi2, vi3, vo1, vo2, decayunits)
nctrains, nctest = hf.load_data_from_nc_as_lists(root_dir)

all_train_input, all_train_output = preload_data_filterT(nctrains, Ntrain, decayunits=decayunits,plot=False)
all_test_input, all_test_output = preload_data_filterT(nctest, Ntest, decayunits=decayunits,plot=False)

#Normalize data
#Compute mean and variance for normalization
mean_input=np.nanmean(np.concatenate((all_train_input, all_test_input), axis=0),axis=(0, 2, 3))
mean_output=np.nanmean(np.concatenate((all_train_output, all_test_output), axis=0),axis=(0, 2, 3))
#Subtract the data with their means
all_train_input=all_train_input-mean_input[None, :, None, None]
all_train_output=all_train_output-mean_output[None, :, None, None]
all_test_input=all_test_input-mean_input[None, :, None, None]
all_test_output=all_test_output-mean_output[None, :, None, None]
#Compute the variances
var_input=np.nanmean((np.concatenate((all_train_input, all_test_input), axis=0))**2,axis=(0, 2, 3))
var_output=np.nanmean((np.concatenate((all_train_output, all_test_output), axis=0))**2,axis=(0, 2, 3))
print("mean and variance of all input data, after lowpass is applied to Temperature:")
print(mean_input,var_input)
print("mean and variance of all output data, after lowpass is applied to Temperature:")
print(mean_output,var_output)

#Scale the data so that they have variance of 1
all_train_input=all_train_input/np.sqrt(var_input[None, :, None, None])
all_train_output=all_train_output/np.sqrt(var_output[None, :, None, None])
all_test_input=all_test_input/np.sqrt(var_input[None, :, None, None])
all_test_output=all_test_output/np.sqrt(var_output[None, :, None, None])
#Have checked that after these operations, the data is scaled to be zero mean and variance 1.


#Recording performance metrics on test data after eaching training cycle
R2_all = np.zeros(nensemble)
corr_all = np.zeros(nensemble)
for iensemble in np.arange(nensemble):
    run_model(var_input_names, var_output_names, save_fn_prefix, N_inp, N_out, iensemble, R2_all, corr_all)  
print('R2 from the best models in each run are:')
print(R2_all)
print('corr from the best models in each run are:')
print(corr_all)




mean and variance of all input data, after lowpass is applied to Temperature:
[ 2.51413189e+01  3.56612545e-02 -1.90587743e-03] [0.33904831 0.04679374 0.04875586]
mean and variance of all output data, after lowpass is applied to Temperature:
[-5.16228102e-04 -9.83592627e-05] [9.36516511e-05 1.01456128e-04]
Model has  1.124706  million params


  0%|          | 1/300 [00:06<30:01,  6.02s/it]

R2: 0.01745378222902305  corr:  0.18168496303851148  pval:  0.0


  1%|          | 2/300 [00:12<29:56,  6.03s/it]

R2: 0.14581026687654242  corr:  0.5080561102791352  pval:  0.0


...

R2: 0.9061665076888952  corr:  0.9519897069081895  pval:  0.0


 83%|████████▎ | 250/300 [21:22<04:21,  5.23s/it]

R2: 0.9066703125338161  corr:  0.9522428879603022  pval:  0.0


100%|██████████| 300/300 [25:26<00:00,  5.09s/it]


training finished
start saving
model saved
R2 from the best models in each run are:
[0.89946387 0.9031889  0.90162463 0.90396157 0.90645028 0.90472382
 0.90471772 0.90314403 0.9031726  0.90666491]
corr from the best models in each run are:
[0.94848566 0.95046133 0.94968621 0.95079382 0.95216776 0.95138964
 0.95117572 0.95035215 0.95037373 0.95223941]


In [11]:
# vi1 = 'T_xy_ins'

# vo1 = 'ssh_cos'
# vo2 = 'ssh_sin'

# batch_size = 100 #maximizing it so that the GPU memory maxes out. Needs to be divisible by Ntrain. Otherwise there will be size mismatch issues.
# lr0 = 0.005*10/batch_size #Roughly should scale inversely to batch_size


# var_input_names = [vi1]
# var_output_names = [vo1, vo2]
# N_inp = len(var_input_names)
# N_out = len(var_output_names)

# save_fn_prefix  = 'any_{}_{}{}_nobatchnorm_degradeT_du_{}'.format(vi1, vo1, vo2, decayunits)
# nctrains, nctest = hf.load_data_from_nc_as_lists(root_dir)

# all_train_input, all_train_output = preload_data_filterT(nctrains, Ntrain, decayunits=decayunits,plot=False)
# all_test_input, all_test_output = preload_data_filterT(nctest, Ntest, decayunits=decayunits,plot=False)

# #Normalize data
# #Compute mean and variance for normalization
# mean_input=np.nanmean(np.concatenate((all_train_input, all_test_input), axis=0),axis=(0, 2, 3))
# mean_output=np.nanmean(np.concatenate((all_train_output, all_test_output), axis=0),axis=(0, 2, 3))
# #Subtract the data with their means
# all_train_input=all_train_input-mean_input[None, :, None, None]
# all_train_output=all_train_output-mean_output[None, :, None, None]
# all_test_input=all_test_input-mean_input[None, :, None, None]
# all_test_output=all_test_output-mean_output[None, :, None, None]
# #Compute the variances
# var_input=np.nanmean((np.concatenate((all_train_input, all_test_input), axis=0))**2,axis=(0, 2, 3))
# var_output=np.nanmean((np.concatenate((all_train_output, all_test_output), axis=0))**2,axis=(0, 2, 3))
# print("mean and variance of all input data, after lowpass is applied to Temperature:")
# print(mean_input,var_input)
# print("mean and variance of all output data, after lowpass is applied to Temperature:")
# print(mean_output,var_output)

# #Scale the data so that they have variance of 1
# all_train_input=all_train_input/np.sqrt(var_input[None, :, None, None])
# all_train_output=all_train_output/np.sqrt(var_output[None, :, None, None])
# all_test_input=all_test_input/np.sqrt(var_input[None, :, None, None])
# all_test_output=all_test_output/np.sqrt(var_output[None, :, None, None])
# #Have checked that after these operations, the data is scaled to be zero mean and variance 1.


# #Recording performance metrics on test data after eaching training cycle
# R2_all = np.zeros(nensemble)
# corr_all = np.zeros(nensemble)
# for iensemble in np.arange(nensemble):
#     run_model(var_input_names, var_output_names, save_fn_prefix, N_inp, N_out, iensemble, R2_all, corr_all)  
# print('R2 from the best models in each run are:')
# print(R2_all)
# print('corr from the best models in each run are:')
# print(corr_all)

In [12]:
# vi1 = 'ssh_ins'
# vi2 = 'T_xy_ins'
# vi3 = 'u_xy_ins'
# vi4 = 'v_xy_ins'

# vo1 = 'ssh_cos'
# vo2 = 'ssh_sin'

# batch_size = 50 #maximizing it so that the GPU memory maxes out. Needs to be divisible by Ntrain. Otherwise there will be size mismatch issues.
# lr0 = 0.005*10/batch_size #Roughly should scale inversely to batch_size

# var_input_names = [vi1, vi2, vi3, vi4]
# var_output_names = [vo1, vo2]
# N_inp = len(var_input_names)
# N_out = len(var_output_names)

# save_fn_prefix  = 'any_{}{}{}{}_{}{}_nobatchnorm_degradeT_du_{}'.format(vi1, vi2, vi3, vi4, vo1, vo2, decayunits)
# nctrains, nctest = hf.load_data_from_nc_as_lists(root_dir)

# all_train_input, all_train_output = preload_data_filterT(nctrains, Ntrain, decayunits=decayunits,plot=False)
# all_test_input, all_test_output = preload_data_filterT(nctest, Ntest, decayunits=decayunits,plot=False)

# #Normalize data
# #Compute mean and variance for normalization
# mean_input=np.nanmean(np.concatenate((all_train_input, all_test_input), axis=0),axis=(0, 2, 3))
# mean_output=np.nanmean(np.concatenate((all_train_output, all_test_output), axis=0),axis=(0, 2, 3))
# #Subtract the data with their means
# all_train_input=all_train_input-mean_input[None, :, None, None]
# all_train_output=all_train_output-mean_output[None, :, None, None]
# all_test_input=all_test_input-mean_input[None, :, None, None]
# all_test_output=all_test_output-mean_output[None, :, None, None]
# #Compute the variances
# var_input=np.nanmean((np.concatenate((all_train_input, all_test_input), axis=0))**2,axis=(0, 2, 3))
# var_output=np.nanmean((np.concatenate((all_train_output, all_test_output), axis=0))**2,axis=(0, 2, 3))
# print("mean and variance of all input data, after lowpass is applied to Temperature:")
# print(mean_input,var_input)
# print("mean and variance of all output data, after lowpass is applied to Temperature:")
# print(mean_output,var_output)

# #Scale the data so that they have variance of 1
# all_train_input=all_train_input/np.sqrt(var_input[None, :, None, None])
# all_train_output=all_train_output/np.sqrt(var_output[None, :, None, None])
# all_test_input=all_test_input/np.sqrt(var_input[None, :, None, None])
# all_test_output=all_test_output/np.sqrt(var_output[None, :, None, None])
# #Have checked that after these operations, the data is scaled to be zero mean and variance 1.


# #Recording performance metrics on test data after eaching training cycle
# R2_all = np.zeros(nensemble)
# corr_all = np.zeros(nensemble)
# for iensemble in np.arange(nensemble):
#     run_model(var_input_names, var_output_names, save_fn_prefix, N_inp, N_out, iensemble, R2_all, corr_all)  
# print('R2 from the best models in each run are:')
# print(R2_all)
# print('corr from the best models in each run are:')
# print(corr_all)

